# Genetic Algorithm 
Reference : https://towardsdatascience.com/evolution-of-a-salesman-a-complete-genetic-algorithm-tutorial-for-python-6fe5d2b3ca35

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, random, operator, time

In [2]:
#City class 구성
class City :
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def distance(self,city):
        Dis = np.sqrt((self.x-city.x)**2+(self.y-city.y)**2)
        return Dis
    
    def __repr__(self):
        return '('+str(self.x)+','+str(self.y)+')'
    

In [3]:
#Fitness
class Fitness :
    def __init__(self,route):
        self.route = route
        self.distance =0
        self.fitness = 0
        
        
        
    def routeDistance(self):
        pathdistance = 0
        if self.distance == 0:
            for i in range(0,len(self.route)):
                if i+1 < len(self.route):
                    fromCity = self.route[i]
                    toCity = self.route[i+1]
                    pathdistance += fromCity.distance(toCity)
                else :
                    fromCity = self.route[len(self.route)-1]
                    toCity = self.route[0]
                    pathdistance += fromCity.distance(toCity)
        return pathdistance
    
    def routeFitness(self):
        routefitness = 0
        routedistance = self.routeDistance()
        if self.fitness==0:
            routefitness = 1/float(routedistance)
            
        return routefitness
              

In [4]:
route1 = []

In [5]:
for i in range(5):
    route1.append(City(random.randint(0,100),random.randint(0,100)))

In [6]:
route1

[(44,70), (71,26), (32,69), (45,7), (36,68)]

In [7]:
fitness1 = Fitness(route1).routeFitness()

In [8]:
fitness1

0.004116409521622737

In [9]:
#createRoute : CityList에서 구성된 랜덤샘플로 route구성
def createRoute(cityList):
    route = random.sample(cityList,len(cityList))
    return route

In [10]:
route1

[(44,70), (71,26), (32,69), (45,7), (36,68)]

In [11]:
citylist1=route1

In [12]:
citylist1

[(44,70), (71,26), (32,69), (45,7), (36,68)]

In [13]:
createRoute(citylist1)

[(36,68), (71,26), (44,70), (32,69), (45,7)]

In [14]:
#cityList,popNum입력받아 popNum개의 population 형성
def initialPopulation(cityList,popNum):
    population = []
    for i in range(popNum):
        population.append(createRoute(cityList))
    return population

In [15]:
a = initialPopulation(citylist1,4)

In [16]:
a

[[(71,26), (36,68), (44,70), (45,7), (32,69)],
 [(36,68), (44,70), (45,7), (32,69), (71,26)],
 [(36,68), (45,7), (32,69), (44,70), (71,26)],
 [(36,68), (45,7), (44,70), (32,69), (71,26)]]

In [17]:
poprank = {}
for i in range(len(a)):
    fitness = Fitness(a[i]).routeFitness()
    poprank[i]=fitness



In [18]:
poprank

{0: 0.004043249181828785,
 1: 0.004043249181828785,
 2: 0.004109382149971834,
 3: 0.004009087166139749}

In [19]:

routerank = sorted(poprank.items(),key = operator.itemgetter(1),reverse=True)


In [20]:
routerank

[(2, 0.004109382149971834),
 (0, 0.004043249181828785),
 (1, 0.004043249181828785),
 (3, 0.004009087166139749)]

In [21]:
df = pd.DataFrame(routerank,columns=['Index','Fitness'])

In [22]:
df

,Index,Fitness
0,2,0.004109
1,0,0.004043
2,1,0.004043
3,3,0.004009


In [23]:
df['Rank']=df.index+1

In [24]:

df['Rank']=df['Rank'].values[::-1]

In [25]:
df

,Index,Fitness,Rank
0,2,0.004109,4
1,0,0.004043,3
2,1,0.004043,2
3,3,0.004009,1


In [26]:
len(df)

4

In [27]:
df['Pressure']=df['Rank']/(len(df)*(len(df)+1)/2)*100
df['Cumsum']=df['Pressure'].cumsum()

In [28]:
df

,Index,Fitness,Rank,Pressure,Cumsum
0,2,0.004109,4,40.0,40.0
1,0,0.004043,3,30.0,70.0
2,1,0.004043,2,20.0,90.0
3,3,0.004009,1,10.0,100.0


In [29]:
#rankRoutes로 population의 route중 fitness좋은것부터
#순서대로 (index,fitness)형태로 출력
def rankRoutes(population):
    poprank = {}
    for i in range(len(population)):
        fitness = Fitness(population[i]).routeFitness()
        poprank[i]=fitness
    return sorted(poprank.items(),key = operator.itemgetter(1),reverse=True)


In [30]:
rankRoutes(a)

[(2, 0.004109382149971834),
 (0, 0.004043249181828785),
 (1, 0.004043249181828785),
 (3, 0.004009087166139749)]

In [31]:
#roulette wheel이용해서 selection. population length만큼 선택하기.
#rank selection
def selection(popRank):
    selected = []
    df = pd.DataFrame(popRank,columns=['Index','Fitness'])
    df['Rank']=df.index+1
    df['Rank']=df['Rank'].values[::-1]
    df['Pressure']=df['Rank']/(len(df)*(len(df)+1)/2)*100
    df['Cumsum']=df['Pressure'].cumsum()
    for i in range(len(popRank)):
        pick = random.random()*100
        for k in range(len(popRank)):
            if pick < df.iloc[k,4]:
                selected.append(popRank[k])
                break
    return selected
                
        
    

In [32]:
df.iloc[1,3]

30.0

In [33]:
df

,Index,Fitness,Rank,Pressure,Cumsum
0,2,0.004109,4,40.0,40.0
1,0,0.004043,3,30.0,70.0
2,1,0.004043,2,20.0,90.0
3,3,0.004009,1,10.0,100.0


In [34]:
routerank

[(2, 0.004109382149971834),
 (0, 0.004043249181828785),
 (1, 0.004043249181828785),
 (3, 0.004009087166139749)]

In [35]:
selc= selection(routerank)

In [36]:
selc

[(2, 0.004109382149971834),
 (2, 0.004109382149971834),
 (0, 0.004043249181828785),
 (1, 0.004043249181828785)]

In [37]:
#selection 결과 바탕으로 matingpool형성
def matingpool(selected,population):
    #population에서 룰렛휠로 selected된 route들로 mating pool 형성
    index = []
    for i in range(len(selected)):
        index.append(selected[i][0])
    pool = []
    for i in range(len(index)):
        pool.append(population[index[i]])
    return pool    
    
    
           

In [38]:
mating = matingpool(selc,a)

In [39]:
mating[0]

[(36,68), (45,7), (32,69), (44,70), (71,26)]

In [42]:
mating

[[(36,68), (45,7), (32,69), (44,70), (71,26)],
 [(36,68), (45,7), (32,69), (44,70), (71,26)],
 [(71,26), (36,68), (44,70), (45,7), (32,69)],
 [(36,68), (44,70), (45,7), (32,69), (71,26)]]

In [ ]:
def PMXbreed(parent1,parent2):
    
    abs1 = random.randint(0,len(parent1))
    abs2 = random.randint(0,len(parent2))
    cxpoint1 = min(abs1,abs2)
    cxpoint2 = max(abs1,abs2)
    child = [None]*len(parent1)
    for i in range(cxpoint1,cxpoint2):
        temp1 = parent1[i]
        temp2 = parent2[i]
        parent1[i]= temp2
        parent2[i]= temp1
        
       

In [62]:
parent1 = mating[0]
parent2 = mating[3]

abs1 = random.randint(0,len(parent1))
abs2 = random.randint(0,len(parent2))
cxpoint1 = min(abs1,abs2)
cxpoint2 = max(abs1,abs2)
child = [None]*len(parent1)

In [63]:
cxpoint1,cxpoint2

(0, 3)

In [64]:
child[cxpoint1:cxpoint2]=parent1[cxpoint1:cxpoint2]
child

[(36,68), (45,7), (32,69), None, None]

In [65]:
for ind,x in enumerate(parent2[cxpoint1:cxpoint2]):
    ind += cxpoint1
    if x not in child:
        while child[ind] != None:
            ind = parent2.index(parent1[ind])
        child[ind]=x

In [67]:
parent1, parent2

([(36,68), (45,7), (32,69), (44,70), (71,26)],
 [(36,68), (44,70), (45,7), (32,69), (71,26)])

In [68]:
child

[(36,68), (45,7), (32,69), (44,70), None]

In [69]:
for ind,x in enumerate(child):
    if x==None:
        child[ind]= parent2[ind]

In [70]:
child

[(36,68), (45,7), (32,69), (44,70), (71,26)]

In [ ]:
def breed(parent1,parent2):
    child1 = []
    abs1 = random.randint(0,len(parent1))
    abs2 = random.randint(0,len(parent2))
    child1 = []
    for i in range(min(abs1,abs2),max(abs1,abs2)):
        child1.append(parent1[i])
    child2 = [val for val in parent2 if val not in child1]
    Child = child1+child2
    return Child

In [ ]:
[0]*3

In [ ]:
pool = matingpool(selc,a)

In [ ]:
pool

In [ ]:
pool[1]

In [ ]:
random.randint(0,3)

In [ ]:
def breedpopulation(pool):
    breedpop = []
    for i in range(len(pool)):
        parent1 = pool[int(random.random()*len(pool))]
        parent2 = pool[int(random.random()*len(pool))]
        breedpop.append(breed(parent1,parent2))
    return breedpop
#pool에서 랜덤한 부모 2명 추출-> breed
#pool length(=popnum)만큼 breedpopulation(자손pop)생성

In [ ]:
brpop = breedpopulation(pool)

In [ ]:
def mutate(breedroute):
    index1 = int(random.random()*len(breedroute))
    index2 = int(random.random()*len(breedroute))
    change1 = breedroute[index1]
    change2 = breedroute[index2]
    breedroute[index1] = change2
    breedroute[index2] = change1
    return breedroute
            
        
    

In [ ]:
def mutatepopulation(breedpop,mutationrate):
    for i in range(len(breedpop)):
        rate = random.random()
        if rate < mutationrate:
            mutate(breedpop[i])
        
    return breedpop

In [ ]:
brpop

In [ ]:
mutatepopulation(brpop,0.3)

In [ ]:
brpop

In [ ]:
def nextGeneration(population,mutationrate):
    rank = rankRoutes(population)
    selected = selection(rank)
    pool = matingpool(selected,population)
    breedpop = breedpopulation(pool)
    nextpop = mutatepopulation(breedpop,mutationrate)
    return nextpop
    
    

In [ ]:
nextGeneration(brpop,0.2)

In [ ]:
def GeneticAlgorithm(cityList,popSize,mutationrate,GenNum):
    start = time.time()
    pop = initialPopulation(cityList,popSize)
    progress = []
    bestroute= pop[rankRoutes(pop)[0][0]]
    Finaldistance = 1/rankRoutes(pop)[0][1]
    progress.append(1/rankRoutes(pop)[0][1])
    print('First Distance :'+str(1/rankRoutes(pop)[0][1]))
    for i in range(GenNum):
        pop = nextGeneration(pop,mutationrate)
        progress.append(1/rankRoutes(pop)[0][1])
        if Finaldistance > 1/rankRoutes(pop)[0][1]:
            bestroute = pop[rankRoutes(pop)[0][0]]
            Finaldistance = 1/rankRoutes(pop)[0][1]
        
    print('Final Distance :'+str(Finaldistance))
    finish = time.time()
    print('Time :' +str(finish-start))
    print(bestroute)
    plt.plot(progress)
    plt.xlabel('Generation')
    plt.ylabel('Distance')
    plt.show()
    

In [ ]:
cityList = []
for i in range(35):
    cityList.append(City(random.randint(0,100),random.randint(0,100)))

In [ ]:
cityList

In [ ]:
GeneticAlgorithm(cityList,50,0.1,5000)